In [ ]:
import time
from copy import deepcopy

import torch
import torch.optim as optim
import torch.nn.functional as F

from dhg import Graph, Hypergraph
from dhg.data import Cora
from dhg.models import HGNN
from dhg.random import set_seed
from dhg.metrics import HypergraphVertexClassificationEvaluator as Evaluator

set_seed(2022)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

## Load Cora Dataset and Build Hypergraph

Cora is a citation network with 2,708 papers (nodes) across 7 classes. Each paper has a 1,433-dim feature vector (bag-of-words).

We convert the citation graph into a hypergraph using **k-hop neighborhoods**: each vertex's 1-hop neighborhood becomes a hyperedge.

In [ ]:
# Load Cora dataset (auto-downloads on first run)
data = Cora()

X, lbl = data["features"], data["labels"]
train_mask = data["train_mask"]
val_mask = data["val_mask"]
test_mask = data["test_mask"]

# Build a graph from the edge list, then convert to hypergraph via 1-hop neighborhoods
G = Graph(data["num_vertices"], data["edge_list"])
HG = Hypergraph.from_graph_kHop(G, k=1)

print(f"Vertices: {data['num_vertices']}, Features: {data['dim_features']}, Classes: {data['num_classes']}")
print(f"Train: {train_mask.sum()}, Val: {val_mask.sum()}, Test: {test_mask.sum()}")
print(f"Hyperedges: {HG.num_e}")

## Define HGNN Model and Training

HGNN is a 2-layer hypergraph neural network using `HGNNConv` layers with a hidden dim of 16.

In [ ]:
# Initialize model, optimizer, and evaluator
net = HGNN(data["dim_features"], 16, data["num_classes"], drop_rate=0.5)
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)
evaluator = Evaluator(["accuracy", "f1_score", {"f1_score": {"average": "micro"}}])

# Move everything to device
X, lbl = X.to(device), lbl.to(device)
HG = HG.to(device)
net = net.to(device)

print(net)

In [ ]:
# Training loop
num_epochs = 500
best_state = None
best_epoch, best_val = 0, 0
train_losses = []

for epoch in range(num_epochs):
    # Train
    net.train()
    optimizer.zero_grad()
    outs = net(X, HG)
    loss = F.cross_entropy(outs[train_mask], lbl[train_mask])
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # Validate
    net.eval()
    with torch.no_grad():
        outs = net(X, HG)
        val_res = evaluator.validate(lbl[val_mask], outs[val_mask])

    if val_res > best_val:
        best_epoch = epoch
        best_val = val_res
        best_state = deepcopy(net.state_dict())

    if epoch % 50 == 0 or epoch == num_epochs - 1:
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f} | Val: {val_res:.4f}")

print(f"\nBest validation: {best_val:.4f} at epoch {best_epoch}")

## Evaluate on Test Set

In [ ]:
# Load best model and evaluate on test set
net.load_state_dict(best_state)
net.eval()
with torch.no_grad():
    outs = net(X, HG)
    test_res = evaluator.test(lbl[test_mask], outs[test_mask])

print(f"Test Results (best model from epoch {best_epoch}):")
print(test_res)

## Training Loss Curve

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(train_losses)
ax.set_xlabel("Epoch")
ax.set_ylabel("Cross-Entropy Loss")
ax.set_title("HGNN Training Loss on Cora")
ax.axvline(best_epoch, color="r", linestyle="--", alpha=0.7, label=f"Best val epoch ({best_epoch})")
ax.legend()
plt.tight_layout()
plt.show()